# Analyses cartographiques

---
Dans ce notebook nous allons explorer des approches pour restituer un ou deux indicateurs sur une carte en fonction de l'analyse souhaitée.

Plusieurs cas de figure sont présentés ci-après.


**Représentation d'un indicateur à une date t**

1. Indicateur numérique

> Carte de chaleur

> Discrétisation selon un seuil et visualisation des catégories sur une carte de couleur


2. Indicateur catégoriel

> Visualisation des catégories sur une carte de couleur


**Représentation de l'évolution d'un indicateur entre deux dates t1 et t2**

1. Indicateur numérique

> Calcul du delta entre les deux dates ($Xt2 - Xt1$) et visualisation du résultat sur une carte de chaleur 

2. Indicateur catégoriel

> Création de nouvelles catégories des évolutions possibles, et visualisation de ces croisements sur une carte de couleur. 

    Exemple : indicateur avec deux catégories "Yes", "No" --> Croisement :
        
        - "Yes - No"
        - "No - Yes"
        - "No" (n'a pas changé, i.e. "No - No")
        - "Yes" (n'a pas changé, i.e. "Yes - Yes")


**Représentation du croisement entre deux indicateurs**

1. Indicateurs numériques

> Définir un seuil pour chaque indicateur

> Créer deux catégories supérieur/inférieur pour chaque indicateur en fonction du seuil

> Croiser les nouvelles catégories des 2 indicateurs (4 catégories résultantes) et les visualiser sur une carte de couleurs

2. Indicateurs catégoriels

> croisement des catégories et visualisation sur uen carte de couleur

3. Indicateurs Catégoriel x Numérique

> Créer deux catégories pour l'indicateur numérique à parir d'un seuil défini

> Croiser les nouvelles catégories de l'indicateur numérique avec celles de l'indicateur catégoriel et visualiser sur une carte de couleur

### Import des librairies

In [1]:
%config IPCompleter.use_jedi = False
from IPython.display import display, HTML


display(HTML(data="""
<style>
    div#notebook-container    { width: 99%; }
    div#menubar-container     { width: 99%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import pathlib
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly_express as px
import pyproj
import shapely
import gc
from ipywidgets import interact, Dropdown, Select, FloatSlider
from pandarallel import pandarallel
import multiprocessing as mp
from tqdm.auto import tqdm

%matplotlib inline

In [3]:
pandarallel.initialize(progress_bar=True, nb_workers=mp.cpu_count())

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


### Chargement des données

Ici nous allons nous concentrer sur la maille 5x5 pour raccourcir les temps de traitements. 

Les données que nous allons utiliser dans ce notebook ont été traitées dans le notebook précédent [`notebook-00-exploration-traitement-donnees.ipynb`](./notebook-00-exploration-traitement-donnees.ipynb)

In [4]:
df_full_5x5_proj = pd.read_pickle("../data/donnees_2022-02-27/df_full_5x5.pklz")

In [6]:
# Liste des régions
list_regions = ["Bretagne", "PACA", "Corse"]

Nous allons récupérer la liste des indicateurs numériques, ainsi que les dates renseignées pour chaque indicateur :

In [7]:
list_indicators_num = sorted(list(df_full_5x5_proj[df_full_5x5_proj.value_num.notnull()].name.unique()))
dict_indicator_dates_num = df_full_5x5_proj[df_full_5x5_proj.name.isin(list_indicators_num)].groupby("name")["date_result"].apply(lambda x: list(set(x))).to_dict()

Nous récupérons aussi la liste d'inidcateurs catégoriels et les dates correspondantes :

In [8]:
list_indicators_categ = sorted(list(set(df_full_5x5_proj.name.dropna()) - set(list_indicators_num)))
dict_indicator_dates_categ = df_full_5x5_proj[df_full_5x5_proj.name.isin(list_indicators_categ)].groupby("name")["date_result"].apply(lambda x: list(set(x))).to_dict()

Pour chaque indicateur catégoriel,on récupère la liste des catégories qui seront utilisées pour déterminer les couleurs des cartes :

In [9]:
dict_categories_orders = dict(df_full_5x5_proj[df_full_5x5_proj.name.isin(list_indicators_categ)].groupby("name")["max_cell_prop_value"].apply(lambda x: sorted(list(set(x)))))
dict_categories_orders

{'Etat DCE Paramètres physico chimiques généraux INPN 5 km': ['1',
  '2',
  '3',
  'U',
  'Unknown'],
 'Etat DCE angiospermes INPN 5 km': ['1',
  '2',
  '3',
  'NA',
  'Not applicable',
  'U',
  'Unknown'],
 'Etat DCE autres paramètres biologiques INPN 5 km': ['ERROR', 'U', 'Unknown'],
 'Etat DCE flore aquatique INPN 5 km': ['1',
  '2',
  '3',
  '4',
  '5',
  'NA',
  'U',
  'Unknown'],
 'Etat DCE hydrologie ou régime tidal INPN 5 km': ['1',
  '2',
  'MonitoredButNotUsed',
  'NA',
  'U',
  'Unknown'],
 'Etat DCE invertébrés benthiques INPN 5 km': ['1',
  '2',
  '3',
  '4',
  '5',
  'NA',
  'U',
  'Unknown'],
 'Etat DCE macro algues INPN 5 km': ['1',
  '2',
  '3',
  '4',
  'NA',
  'Not applicable',
  'U',
  'Unknown'],
 'Etat DCE macrophytes INPN 5 km': ['1',
  '3',
  '4',
  'ERROR',
  'N',
  'Not applicable',
  'U',
  'Unknown'],
 'Etat DCE phytobenthos INPN 5 km': ['1',
  '2',
  '3',
  '4',
  '5',
  'NA',
  'Not applicable',
  'U',
  'Unknown'],
 'Etat DCE phytoplancton INPN 5 km': ['1

### Représentation d'un indicateur numérique à une date t

In [10]:
indicator_id_dp = Dropdown(options=dict_indicator_dates_num.keys())
date_dp = Dropdown()
def update_date_options(*args):
    date_dp.options = sorted(dict_indicator_dates_num[indicator_id_dp.value])
indicator_id_dp.observe(update_date_options)

region_dp = Dropdown(options=list_regions, value="Bretagne")

@interact(id_indicator=indicator_id_dp, date=date_dp, region=region_dp)
def plot_indicator_heatmap_interactive(id_indicator, date, region):
    
    tmp_df = df_full_5x5_proj[(df_full_5x5_proj.name==id_indicator) & (df_full_5x5_proj.date_result==date)]\
        .copy()\
        .dropna(axis=0, subset=["value_num"])
    if len(tmp_df) == 0:
        print("-------------Empty dataframe")
        return
    tmp_df.set_index("id_cell", inplace=True)
    title = tmp_df.name.values[0] + " --- " + date
    
    center = {"lat": 46.227638, "lon": 2.213749}
    if region == "Bretagne":
        center = {"lat": 48.202047, "lon": -2.932644}
    if region == "PACA":
        center = {"lat": 43.9351691, "lon": 6.0679194}
    if region == "Corse":
        center = {"lat": 42.039604, "lon": 9.012893}
    
    zoom = 5 if region == "" else 8

    fig = px.choropleth_mapbox(tmp_df, 
                        width=1600, height=800,
                        geojson=tmp_df.geometry,
                        locations=tmp_df.index,
                        color="value_num", hover_name=tmp_df.index,
                        title=title,
                        hover_data=["value_num", "date_result", "content"],
                        mapbox_style="open-street-map",
                       center=center,
                       opacity=1.0,
                       zoom=zoom
                          )
    fig.update_geos(fitbounds="locations", visible=True)
    fig.update_layout(title_x=0.5, margin={"r":0,"l":0,"b":0})
    fig.show(renderer="notebook")
    del tmp_df
    gc.collect()
    print(title)


interactive(children=(Dropdown(description='id_indicator', options=("Aires d'alimentation de captage (surface)…

### Représentation d'un indicateur numérique à une date t, selon un seuil défini

In [12]:
start_indicator = "Maximum Nitrates sous secteurs hydro 5 km"
indicator_id_dp = Dropdown(options=dict_indicator_dates_num.keys(), value=start_indicator)
date_dp = Dropdown(options=sorted(dict_indicator_dates_num[start_indicator]), value=sorted(dict_indicator_dates_num[start_indicator])[0])
region_dp = Dropdown(options=list_regions, value="Bretagne")
float_slider = FloatSlider()

def update_date_options(*args):
    date_dp.options = sorted(dict_indicator_dates_num[indicator_id_dp.value])
    
indicator_id_dp.observe(update_date_options)


def update_ranges(*args):
    tmp_df = df_full_5x5_proj[(df_full_5x5_proj.name==indicator_id_dp.value) & (df_full_5x5_proj.date_result==date_dp.value)]
    _max = tmp_df.value_num.max()
    _min = tmp_df.value_num.min()
    float_slider.value = (_max-_min)/2
    float_slider.min = _min
    float_slider.max = _max
    float_slider.step = (_max-_min)/10
    
indicator_id_dp.observe(update_ranges)
date_dp.observe(update_ranges)


@interact(id_indicator=indicator_id_dp, date=date_dp, region=region_dp, threshold=float_slider)
def plot_indicator_threshold(id_indicator, date, region, threshold):
    
    tmp_df = df_full_5x5_proj[(df_full_5x5_proj.name==id_indicator) & (df_full_5x5_proj.date_result==date)]\
        .copy()\
        .dropna(axis=0, subset=["value_num"])
    if len(tmp_df) == 0:
        print("-------------Empty dataframe")
        return
    tmp_df.set_index("id_cell", inplace=True)
    tmp_df["threshold"] = tmp_df.value_num.apply(lambda x: "High" if x > threshold else "Low")
    
    title = tmp_df.name.values[0] + " --- " + date + f" (> {threshold})"
    
    center = {"lat": 46.227638, "lon": 2.213749}
    if region == "Bretagne":
        center = {"lat": 48.202047, "lon": -2.932644}
    if region == "PACA":
        center = {"lat": 43.9351691, "lon": 6.0679194}
    if region == "Corse":
        center = {"lat": 42.039604, "lon": 9.012893}
    
    zoom = 5 if region == "" else 8
  
    fig = px.choropleth_mapbox(tmp_df, 
                        width=1600, height=800,
                        geojson=tmp_df.geometry,
                        locations=tmp_df.index,
                        color="threshold", hover_name=tmp_df.index,
                        title=title,
                        hover_data=["threshold", "date_result", "content"],
                        mapbox_style="open-street-map",
                       center=center,
                       opacity=1,
                       zoom=zoom,
                          )
    fig.update_geos(fitbounds="locations", visible=True)
    fig.update_layout(title_x=0.5, margin={"r":0,"l":0,"b":0})
    fig.show(renderer="notebook")
    del tmp_df
    gc.collect()
    print(title)
    

interactive(children=(Dropdown(description='id_indicator', index=24, options=("Aires d'alimentation de captage…

### Représentation d'un indicateur catégoriel à une date t

In [13]:
indicator_id_dp_str = Dropdown(options=dict_indicator_dates_categ.keys())
date_dp_str = Dropdown()
def update_date_options_str(*args):
    date_dp_str.options = sorted(dict_indicator_dates_categ[indicator_id_dp_str.value])
indicator_id_dp_str.observe(update_date_options_str)

region_dp = Dropdown(options=list_regions, value="Bretagne")

@interact(id_indicator=indicator_id_dp_str, date=date_dp_str, region=region_dp)
def plot_indicator_colormap_interactive(id_indicator, date, region):

    tmp_df = df_full_5x5_proj[(df_full_5x5_proj.name==id_indicator) & (df_full_5x5_proj.date_result==date)]\
        .copy()\
        .dropna(axis=0, subset=["value_str"])
    if len(tmp_df) == 0:
        print("-------------Empty dataframe")
        return
    tmp_df.set_index("id_cell", inplace=True)
    title = tmp_df.name.values[0] + " --- " + date
    
    center = {"lat": 46.227638, "lon": 2.213749}
    if region == "Bretagne":
        center = {"lat": 48.202047, "lon": -2.932644}
    if region == "PACA":
        center = {"lat": 43.9351691, "lon": 6.0679194}
    if region == "Corse":
        center = {"lat": 42.039604, "lon": 9.012893}
    
    zoom = 5 if region == "" else 8
    
    fig = px.choropleth_mapbox(tmp_df, 
                        width=1600, height=800,
                        geojson=tmp_df.geometry,
                        locations=tmp_df.index,
                        color="max_cell_prop_value", hover_name=tmp_df.index,
                        title=title,
                        hover_data=["max_cell_prop_value", "date_result", "content"],
                        mapbox_style="open-street-map",
                       center=center,
                       opacity=1,
                       zoom=zoom,
                       category_orders={"max_cell_prop_value": dict_categories_orders[id_indicator]}
                          )
    fig.update_geos(fitbounds="locations", visible=True)
    fig.update_layout(title_x=0.5, margin={"r":0,"l":0,"b":0})
    fig.show(renderer="notebook")
    del tmp_df
    gc.collect()
    print(title)


interactive(children=(Dropdown(description='id_indicator', options=('Etat DCE Paramètres physico chimiques gén…

### Représentation évolution d'un indicateur numérique entre deux dates t1 et t2

In [14]:
%%time

# Conversion de la colonne des dates de string vers un objet datetime
df_full_5x5_proj["date_result_dt"] = pd.to_datetime(df_full_5x5_proj.date_result)

CPU times: user 184 ms, sys: 16.2 ms, total: 201 ms
Wall time: 206 ms


In [15]:
indicator_id_dp = Dropdown(options=dict_indicator_dates_num.keys(), value=start_indicator)
region_dp = Dropdown(options=list_regions, value="Bretagne")

@interact(id_indicator=indicator_id_dp, region=region_dp)
def plot_indicator_heatmap_evolution(id_indicator, region):
    
    tmp_df = df_full_5x5_proj[(df_full_5x5_proj.name==id_indicator)]\
        .copy()\
        .dropna(axis=0, subset=["value_num"])
    if len(tmp_df) == 0:
        print("-------------Empty dataframe")
        return
    tmp_df.set_index("id_cell", inplace=True)
    
    date_min, date_max = tmp_df.date_result_dt.min(), tmp_df.date_result_dt.max()
    if date_min == date_max:
        print("Single date: ", date_min)
        return
    
    title = "Evolution " + tmp_df.name.values[0] + " : "+ str(date_min)+" --> "+str(date_max)
    
    def delta_df(df):
        try:
            return df[df.date_result_dt == date_max].value_num.values[0] - df[df.date_result_dt == date_min].value_num.values[0]
        except:
            return np.nan
    
    tmp_df_delta = tmp_df.groupby("id_cell").apply(lambda df: delta_df(df)).reset_index().rename({0: "Values"}, axis=1)
    tmp_df = tmp_df[["geometry", "content"]].merge(tmp_df_delta, on="id_cell", how="left")
    
    center = {"lat": 46.227638, "lon": 2.213749}
    if region == "Bretagne":
        center = {"lat": 48.202047, "lon": -2.932644}
    if region == "PACA":
        center = {"lat": 43.9351691, "lon": 6.0679194}
    if region == "Corse":
        center = {"lat": 42.039604, "lon": 9.012893}
    
    zoom = 5 if region == "" else 8
        
    fig = px.choropleth_mapbox(tmp_df, 
                        width=1600, height=800,
                        geojson=tmp_df.geometry,
                        locations=tmp_df.index,
                        color="Values", hover_name=tmp_df.index,
                        title=title,
                        hover_data=["Values", "content"],
                        mapbox_style="open-street-map",
                       center=center,
                       opacity=1,
                       zoom=zoom,
                          )
    fig.update_geos(fitbounds="locations", visible=True)
    fig.update_layout(title_x=0.5, margin={"r":0,"l":0,"b":0})
    fig.show(renderer="notebook")
    del tmp_df
    gc.collect()
    print(title)

interactive(children=(Dropdown(description='id_indicator', index=24, options=("Aires d'alimentation de captage…

### Représentation de l'évolution d'un indicateur catégoriel entre deux dates t1 et t2

In [16]:
indicator_id_dp_str = Dropdown(options=dict_indicator_dates_categ.keys())

region_dp = Dropdown(options=list_regions, value="Bretagne")

@interact(id_indicator=indicator_id_dp_str, region=region_dp)
def plot_indicator_colormap_evolution(id_indicator, region):
    
    # calculer le delta entre deux dates
    def delta_df_categorical(df):
        try:
            value_t1 = df[df.date_result_dt == date_min].max_cell_prop_value.values[0]
            value_t2 = df[df.date_result_dt == date_max].max_cell_prop_value.values[0]
            delta = f"{value_t1} --> {value_t2}" if value_t1 != value_t2 else value_t1
            return delta
        except:
            return None

    tmp_df = df_full_5x5_proj[(df_full_5x5_proj.name==id_indicator)]\
        .copy()\
        .dropna(axis=0, subset=["value_str"])
    if len(tmp_df) == 0:
        print("-------------Empty dataframe")
        return
    tmp_df.set_index("id_cell", inplace=True)
    
    
    date_min, date_max = tmp_df.date_result_dt.min(), tmp_df.date_result_dt.max()
    if date_min == date_max:
        print("Single date: ", date_min)
        return
    
    title = "Evolution " + tmp_df.name.values[0] + " : "+ str(date_min)+" --> "+str(date_max)
    
    tmp_df_delta = tmp_df.groupby("id_cell").apply(lambda df: delta_df_categorical(df)).reset_index().rename({0: "Values"}, axis=1)
    tmp_df = tmp_df[["geometry", "content"]].merge(tmp_df_delta, on="id_cell", how="left")
    tmp_df = tmp_df.dropna(axis=0, subset=["Values"])
    
    center = {"lat": 46.227638, "lon": 2.213749}
    if region == "Bretagne":
        center = {"lat": 48.202047, "lon": -2.932644}
    if region == "PACA":
        center = {"lat": 43.9351691, "lon": 6.0679194}
    if region == "Corse":
        center = {"lat": 42.039604, "lon": 9.012893}
    
    zoom = 5 if region == "" else 8
    
    fig = px.choropleth_mapbox(tmp_df, 
                        width=1600, height=800,
                        geojson=tmp_df.geometry,
                        locations=tmp_df.index,
                        color="Values", hover_name=tmp_df.index,
                        title=title,
                        hover_data=["Values", "content"],
                        mapbox_style="open-street-map",
                       center=center,
                       opacity=1,
                       zoom=zoom,
                          )
    fig.update_geos(fitbounds="locations", visible=True)
    fig.update_layout(title_x=0.5, margin={"r":0,"l":0,"b":0})
    fig.show(renderer="notebook")
    del tmp_df
    gc.collect()
    print(title)


interactive(children=(Dropdown(description='id_indicator', options=('Etat DCE Paramètres physico chimiques gén…

### Représentation du croisement entre deux indicateurs numériques

In [17]:
start_indicator = "Maximum Nitrates sous secteurs hydro 5 km"
start_indicator2 = "Maximum Glyphosate sous secteurs hydro 5 km"
indicator_id_dp = Dropdown(options=dict_indicator_dates_num.keys(), value=start_indicator)
indicator_id_dp2 = Dropdown(options=dict_indicator_dates_num.keys(), value=start_indicator2)
start_dates = sorted(list(set(dict_indicator_dates_num[start_indicator]).intersection(set(dict_indicator_dates_num[start_indicator2]))))
date_dp = Dropdown(options=start_dates, value=start_dates[0])
region_dp = Dropdown(options=list_regions, value="Bretagne")
float_slider = FloatSlider()
float_slider2 = FloatSlider()

def update_date_options(*args):
    date_dp.options = sorted(list(set(dict_indicator_dates_num[indicator_id_dp.value]).intersection(set(dict_indicator_dates_num[indicator_id_dp2.value]))))
    
indicator_id_dp.observe(update_date_options)
indicator_id_dp2.observe(update_date_options)


def update_ranges(*args):
    tmp_df = df_full_5x5_proj[(df_full_5x5_proj.name==indicator_id_dp.value) & (df_full_5x5_proj.date_result==date_dp.value)]
    _max = tmp_df.value_num.max()
    _min = tmp_df.value_num.min()
    float_slider.value = (_max-_min)/2
    float_slider.min = _min
    float_slider.max = _max
    float_slider.step = (_max-_min)/10
    
    tmp_df = df_full_5x5_proj[(df_full_5x5_proj.name==indicator_id_dp2.value) & (df_full_5x5_proj.date_result==date_dp.value)]
    _max = tmp_df.value_num.max()
    _min = tmp_df.value_num.min()
    float_slider2.value = (_max-_min)/2
    float_slider2.min = _min
    float_slider2.max = _max
    float_slider2.step = (_max-_min)/10
    
indicator_id_dp.observe(update_ranges)
indicator_id_dp2.observe(update_ranges)
date_dp.observe(update_ranges)


@interact(id_indicator=indicator_id_dp, id_indicator2=indicator_id_dp2, date=date_dp, region=region_dp, threshold=float_slider, threshold2=float_slider2)
def plot_cross_indicator_num(id_indicator, id_indicator2, date, region, threshold, threshold2):
    
    tmp_df = df_full_5x5_proj[(df_full_5x5_proj.name.isin([id_indicator, id_indicator2])) & (df_full_5x5_proj.date_result==date)]\
        .copy()\
        .dropna(axis=0, subset=["value_num"])
    if len(tmp_df) == 0:
        print("-------------Empty dataframe")
        return
    
    tmp_df["cross_values"] = None
    tmp_df.loc[tmp_df.name==id_indicator, "cross_values"] = tmp_df[tmp_df.name==id_indicator].value_num.apply(lambda x: "High" if x > threshold else "Low")
    tmp_df.loc[tmp_df.name==id_indicator2, "cross_values"] = tmp_df[tmp_df.name==id_indicator2].value_num.apply(lambda x: "High" if x > threshold2 else "Low")
    
    # calculer le delta entre deux dates
    def cross_df_categorical(df):
        try:
            value_t1 = df[df["name"] == id_indicator].cross_values.values[0]
            value_t2 = df[df["name"] == id_indicator2].cross_values.values[0]
            delta = f"'{value_t1}' X '{value_t2}'"
            return delta
        except Exception as e:
            return None
    
    cells_to_keep = tmp_df.groupby("id_cell")["name"].apply(len)
    cells_to_keep = cells_to_keep[cells_to_keep == 2].index.tolist()
    tmp_df = tmp_df[tmp_df.id_cell.isin(cells_to_keep)]
    tmp_df_delta = tmp_df.groupby("id_cell").apply(lambda df: cross_df_categorical(df)).reset_index().rename({0: "cross_values"}, axis=1)
    tmp_df = tmp_df[["id_cell", "geometry", "content", "date_result"]].merge(tmp_df_delta, on="id_cell", how="left")
    tmp_df = tmp_df.dropna(axis=0, subset=["cross_values"])
    
    title = f"{id_indicator} / {id_indicator2}" + " --- " + date
    
    center = {"lat": 46.227638, "lon": 2.213749}
    if region == "Bretagne":
        center = {"lat": 48.202047, "lon": -2.932644}
    if region == "PACA":
        center = {"lat": 43.9351691, "lon": 6.0679194}
    if region == "Corse":
        center = {"lat": 42.039604, "lon": 9.012893}
    
    zoom = 5 if region == "" else 8  
    
    tmp_df.set_index("id_cell", inplace=True)
    fig = px.choropleth_mapbox(tmp_df, 
                        width=1600, height=800,
                        geojson=tmp_df.geometry,
                        locations=tmp_df.index,
                        color="cross_values", hover_name=tmp_df.index,
                        title=title,
                        hover_data=["cross_values", "date_result", "content"],
                        mapbox_style="open-street-map",
                       center=center,
                       opacity=1.0,
                       zoom=zoom,
                          )
    fig.update_geos(fitbounds="locations", visible=True)
    fig.update_layout(title_x=0.5, margin={"r":0,"l":0,"b":0})
    fig.show(renderer="notebook")
    del tmp_df
    gc.collect()
    print(title)
    

interactive(children=(Dropdown(description='id_indicator', index=24, options=("Aires d'alimentation de captage…